In [ ]:
import http.client, urllib.request, urllib.parse, urllib.error, base64, json
import pandas as pd
metro_api_key = {'api_key':'b62fe5a2b0be42a688390dd6916e040a'}

def code_request(line_code):
    params = urllib.parse.urlencode({
        # Request parameters
        'LineCode': line_code,
    })

    try:
        conn = http.client.HTTPSConnection('api.wmata.com')
        conn.request("GET", "/Rail.svc/json/jStations?%s" % params, "{body}", metro_api_key)
        response = conn.getresponse()
        data = response.read()
        red = json.loads(data)
        conn.close()
        return pd.DataFrame(red)['Stations'].apply(lambda x: [x['Lat'], x['Lon'], x['Name'], x['Code']])
    except Exception as e:
        print("[Errno {0}] {1}".format(e.errno, e.strerror))
# [x['Code'],x['Name'],x['Zip']
red = pd.DataFrame(code_request('RD').tolist(), columns=['Lat','Lon', 'Name', 'Code'])
yellow = pd.DataFrame(code_request('YL').tolist(), columns=['Lat','Lon', 'Name', 'Code'])
green = pd.DataFrame(code_request('GR').tolist(), columns=['Lat','Lon', 'Name', 'Code'])
blue = pd.DataFrame(code_request('BL').tolist(), columns=['Lat','Lon', 'Name', 'Code'])
orange = pd.DataFrame(code_request('OR').tolist(), columns=['Lat','Lon', 'Name', 'Code'])
silver = pd.DataFrame(code_request('SV').tolist(), columns=['Lat','Lon', 'Name', 'Code'])

subways = pd.concat([red,yellow,green,blue,orange,silver]).drop_duplicates().reset_index()
subways.to_pickle('subway_locations')